---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    train.csv - the training set (all tickets issued 2004-2011)
    test.csv - the test set (all tickets issued 2012-2016)
    addresses.csv & latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt

def blight_model():
    
    # Your code here
    
    return # Your answer here

df = pd.read_csv('train.csv', encoding = 'ISO-8859-1')
test = pd.read_csv('test.csv', encoding = 'ISO-8859-1')
address = pd.read_csv('addresses.csv', encoding = 'ISO-8859-1')
latlon =  pd.read_csv('latlons.csv', encoding = 'ISO-8859-1')
address_new = address.merge(latlon, on='address', how='inner')
df = df.merge(address_new, on='ticket_id', how='inner')
df = df.set_index('ticket_id')
df_test = test.merge(address_new, on='ticket_id', how='inner')
df_test = df_test.set_index('ticket_id')


In [164]:
df=df.dropna(subset=['compliance'])
df['date_diff'] = (pd.to_datetime(df['hearing_date']) - pd.to_datetime(df['ticket_issued_date'])).dt.days
df_test['date_diff'] = (pd.to_datetime(df_test['hearing_date']) - pd.to_datetime(df_test['ticket_issued_date'])).dt.days
df0=df.where(df.compliance==0)
df1=df.where(df.compliance==1)
keep_list=['agency_name', 'lat', 'lon', 'date_diff', 'violation_code', 
           'disposition', 'fine_amount', 'late_fee', 'discount_amount', 'compliance']
df_test['compliance']=0.0
df = df[keep_list]
df_test = df_test[keep_list]
df = df.dropna()

159880
159880


# Reason for the keep list:

1) admin_fee, state_fee, there is no difference;

2) fine_amount and judgment_amount amount is highly correlated, 0.999


In [165]:
numeric_list = ['lat', 'lon', 'date_diff',  'fine_amount','admin_fee', 'state_fee', 'late_fee', 'discount_amount',
            'clean_up_cost', 'judgment_amount']
non_numeric_list = ['agency_name', 'violation_code', 'disposition']
train_num = len(df)
df_all = pd.concat([df, df_test])
df_all = pd.get_dummies(df_all, columns=non_numeric_list)

In [166]:
df_train = df_all.iloc[:train_num,:]
df_test = df_all.iloc[train_num:, :]

# Feature selection
1) Feature selection based on correlation (Preliminary)

2) Univariate feature selection (select K-best)

3) Random Forest

4) Lasso

5) RFE



Index(['lat', 'lon', 'date_diff', 'fine_amount', 'late_fee', 'discount_amount',
       'compliance',
       'agency_name_Buildings, Safety Engineering & Env Department',
       'agency_name_Department of Public Works',
       'agency_name_Detroit Police Department',
       ...
       'violation_code_9-1-83 - (Dwelling)',
       'violation_code_9-1-83 - (Structures',
       'disposition_Responsible (Fine Waived) by Admis',
       'disposition_Responsible (Fine Waived) by Deter',
       'disposition_Responsible - Compl/Adj by Default',
       'disposition_Responsible - Compl/Adj by Determi',
       'disposition_Responsible by Admission',
       'disposition_Responsible by Default',
       'disposition_Responsible by Determination',
       'disposition_Responsible by Dismissal'],
      dtype='object', length=243)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df_train.drop(columns=['compliance']), df_train['compliance'])
X_train['lon'] = -1 * X_train['lon']

In [218]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.preprocessing import MinMaxScaler
MinMaxScaler().fit_transform(X_train)

X_train[X_train < 0] =0

select_feature = SelectKBest(chi2, k=10).fit(X_train, y_train)

In [219]:
mask = select_feature.get_support() #list of booleans
new_features = [] # The list of your K best features

for bool, feature in zip(mask, X_train.columns):
    if bool:
        new_features.append(feature)
new_features


['date_diff',
 'fine_amount',
 'late_fee',
 'discount_amount',
 'agency_name_Detroit Police Department',
 'violation_code_9-1-81(a)',
 'disposition_Responsible (Fine Waived) by Deter',
 'disposition_Responsible by Admission',
 'disposition_Responsible by Default',
 'disposition_Responsible by Determination']

In [217]:
from sklearn.feature_selection import f_classif
from sklearn.preprocessing import MinMaxScaler
MinMaxScaler().fit_transform(X_train)

X_train[X_train < 0] =0

select_feature = SelectKBest(f_classif, k=10).fit(X_train, y_train)
mask = select_feature.get_support() #list of booleans
new_features = [] # The list of your K best features

for bool, feature in zip(mask, X_train.columns):
    if bool:
        new_features.append(feature)
new_features



/anaconda3/lib/python3.6/site-packages/sklearn/feature_selection/univariate_selection.py:113: UserWarning: Features [ 10  11  16  17  33  35  39  40  43  52  56  57  61  63  69  71  75  78
  80  81  82  84  86  89  92  94  95 102 105 115 121 122 123 124 142 143
 163 173 175 182 192 205 211 214 222 234 236 237 241] are constant.
  UserWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/feature_selection/univariate_selection.py:114: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


['fine_amount',
 'late_fee',
 'discount_amount',
 'agency_name_Buildings, Safety Engineering & Env Department',
 'agency_name_Department of Public Works',
 'violation_code_9-1-81(a)',
 'disposition_Responsible (Fine Waived) by Deter',
 'disposition_Responsible by Admission',
 'disposition_Responsible by Default',
 'disposition_Responsible by Determination']

In [216]:
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import LinearSVC
from sklearn.preprocessing import MinMaxScaler
MinMaxScaler().fit_transform(X_train)
lsvc = LinearSVC(C=0.002, penalty="l1", dual=False).fit(X_train, y_train)
model = SelectFromModel(lsvc, prefit=True)
feature_idx = model.get_support()
X_train.columns[feature_idx]


Index(['lat', 'lon', 'date_diff', 'fine_amount', 'late_fee', 'discount_amount',
       'agency_name_Buildings, Safety Engineering & Env Department',
       'agency_name_Health Department', 'violation_code_9-1-36(a)',
       'disposition_Responsible by Default',
       'disposition_Responsible by Determination'],
      dtype='object')

In [212]:
from sklearn.ensemble import ExtraTreesClassifier

clf = ExtraTreesClassifier().fit(X_train, y_train)
model = SelectFromModel(clf, prefit=True)
X_train.columns[model.get_support()]

Index(['lat', 'lon', 'date_diff', 'fine_amount', 'late_fee', 'discount_amount',
       'violation_code_9-1-36(a)',
       'disposition_Responsible (Fine Waived) by Deter',
       'disposition_Responsible by Admission',
       'disposition_Responsible by Default',
       'disposition_Responsible by Determination'],
      dtype='object')

In [214]:
from sklearn.ensemble import RandomForestClassifier
clf_rf_5 = RandomForestClassifier()      
clr_rf_5 = clf_rf_5.fit(X_train,y_train)
importances = clr_rf_5.feature_importances_
indices = np.argsort(importances)[::-1]
print("Feature ranking:")

for f in range(X_train.shape[1]):
    print(f + 1, X_train.columns[indices[f]], importances[indices[f]])



Feature ranking:
1 lat 0.27613476862614345
2 lon 0.2693518584768267
3 date_diff 0.1728361888398383
4 late_fee 0.09781288039445528
5 discount_amount 0.04620257492108651
6 disposition_Responsible by Default 0.03949381597530849
7 disposition_Responsible by Determination 0.027474403585084885
8 disposition_Responsible by Admission 0.019578287223312875
9 fine_amount 0.013342712656465463
10 disposition_Responsible (Fine Waived) by Deter 0.006299764117236356
11 violation_code_9-1-81(a) 0.002073446884327956
12 agency_name_Department of Public Works 0.0020498593877903686
13 agency_name_Health Department 0.001865183624150997
14 agency_name_Detroit Police Department 0.0015344147530327306
15 agency_name_Buildings, Safety Engineering & Env Department 0.0015138314187358483
16 violation_code_9-1-104 0.0014435952181823353
17 violation_code_22-2-88 0.0014285056458681951
18 violation_code_22-2-61 0.0013613392907187237
19 violation_code_9-1-103(C) 0.0013252164453969573
20 violation_code_9-1-36(a) 0.001254

In [202]:
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier
# Create the RFE object and rank each pixel
clf_rf_3 = RandomForestClassifier()      
rfe = RFE(estimator=clf_rf_3, n_features_to_select=10, step=1)
rfe = rfe.fit(X_train, y_train)
X_train.columns[rfe.support_]

Index(['lat', 'lon', 'date_diff', 'fine_amount', 'late_fee', 'discount_amount',
       'agency_name_Department of Public Works',
       'disposition_Responsible by Admission',
       'disposition_Responsible by Default',
       'disposition_Responsible by Determination'],
      dtype='object')

In [ ]:
from sklearn.feature_selection import RFECV
from sklearn.ensemble import RandomForestClassifier
# Create the RFE object and rank each pixel
clf_rf_3 = RandomForestClassifier()      
rfe = RFECV(estimator=clf_rf_3, cv=5,scoring='accuracy')
rfe = rfe.fit(X_train, y_train)
X_train.columns[rfe.support_]